In [1]:
import pandas as pd
import numpy as np

from datetime import timedelta, datetime
import pytz

from sentimentlabeller import SentimentLabeller

In [2]:
csv_file = '../data/twitter/all_combined_news.csv'
news = pd.read_csv(csv_file)
news

/home/see/anaconda3/envs/env_torch/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3145: DtypeWarning: Columns (4,10,11,12,19) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,url,date,content,renderedContent,id,user,outlinks,tcooutlinks,replyCount,retweetCount,...,quoteCount,conversationId,lang,source,sourceUrl,sourceLabel,media,retweetedTweet,quotedTweet,mentionedUsers
0,https://twitter.com/BBCBusiness/status/5312325...,2014-11-08 23:51:19+00:00,Why capitalism hasn't triumphed http://t.co/mW...,Why capitalism hasn't triumphed bbc.in/1yjTkfs,531232529110802432,"{'username': 'BBCBusiness', 'displayname': 'BB...",['http://bbc.in/1yjTkfs'],['http://t.co/mWniB1M1zY'],2.0,21.0,...,0,531232529110802432,en,"<a href=""http://www.socialflow.com"" rel=""nofol...",http://www.socialflow.com,SocialFlow,NaN,NaN,NaN,NaN
1,https://twitter.com/BBCBusiness/status/5310008...,2014-11-08 08:30:31+00:00,LISTEN: ITB Kilkenomics - economics and comedy...,LISTEN: ITB Kilkenomics - economics and comedy...,531000802685779970,"{'username': 'BBCBusiness', 'displayname': 'BB...",['http://bbc.in/1yh7MF4'],['http://t.co/YnoYcRQoZS'],1.0,11.0,...,0,531000802685779970,en,"<a href=""http://www.socialflow.com"" rel=""nofol...",http://www.socialflow.com,SocialFlow,NaN,NaN,NaN,NaN
2,https://twitter.com/BBCBusiness/status/5309078...,2014-11-08 02:21:15+00:00,LISTEN: Business Matters: Raids shut down 'dar...,LISTEN: Business Matters: Raids shut down 'dar...,530907875041091584,"{'username': 'BBCBusiness', 'displayname': 'BB...",['http://bbc.in/1ACBJF1'],['http://t.co/7p4j7msQbr'],0.0,11.0,...,0,530907875041091584,en,"<a href=""http://www.socialflow.com"" rel=""nofol...",http://www.socialflow.com,SocialFlow,NaN,NaN,NaN,NaN
3,https://twitter.com/BBCBusiness/status/9207536217,2010-02-16 23:30:09+00:00,Sir Paul McCartney says he hopes the famous Ab...,Sir Paul McCartney says he hopes the famous Ab...,9207536217,"{'username': 'BBCBusiness', 'displayname': 'BB...",[],[],0.0,0.0,...,0,9207536217,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://twitter.com,Twitter Web Client,NaN,NaN,NaN,NaN
4,https://twitter.com/BBCBusiness/status/9201211352,2010-02-16 20:50:09+00:00,A former stockbroker used secret and confident...,A former stockbroker used secret and confident...,9201211352,"{'username': 'BBCBusiness', 'displayname': 'BB...",[],[],0.0,1.0,...,0,9201211352,en,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",http://twitter.com,Twitter Web Client,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1105294,https://twitter.com/WSJbusiness/status/3779684...,2013-09-12 01:34:24+00:00,Pandora Media Taps McAndrews as New CEO http:/...,Pandora Media Taps McAndrews as New CEO on.wsj...,377968404021260288,"{'username': 'WSJbusiness', 'displayname': 'WS...",['http://on.wsj.com/182Aiev'],['http://t.co/E8pZfgk1fb'],1.0,2.0,...,0,377968404021260288,pt,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,NaN,NaN,NaN,NaN
1105295,https://twitter.com/WSJbusiness/status/3779573...,2013-09-12 00:50:24+00:00,Michael Dell's Tougher Road Ahead http://t.co/...,Michael Dell's Tougher Road Ahead on.wsj.com/1...,377957332342161409,"{'username': 'WSJbusiness', 'displayname': 'WS...",['http://on.wsj.com/182r1TO'],['http://t.co/2v0zeDAghw'],0.0,4.0,...,0,377957332342161409,en,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,NaN,NaN,NaN,NaN
1105296,https://twitter.com/WSJbusiness/status/3779573...,2013-09-12 00:50:24+00:00,Companies Act on Rising Interest Rates http://...,Companies Act on Rising Interest Rates on.wsj....,377957331394641920,"{'username': 'WSJbusiness', 'displayname': 'WS...",['http://on.wsj.com/182r3el'],['http://t.co/O8D3SeXTyJ'],0.0,1.0,...,0,377957331394641920,en,"<a href=""http://twitterfeed.com"" rel=""nofollow...",http://twitterfeed.com,twitterfeed,NaN,NaN,NaN,NaN
1105297,https://twitter.com/WSJbusiness/status/3779573...,2013-09-12 00:50:24+00:00,Verizon Sells a Record $49 Billion of Bonds ht...,Verizon Sells a Record $49 Billion of Bonds on...,377957329389359105,"{'username': 'WSJbusiness', 'displaynam

In [20]:
sl = SentimentLabeller()
eg = news.renderedContent[0]
sl.label_text(eg, mode='num')

0

In [7]:
# news['sentiment_score'] = news['renderedContent'].apply(lambda x: sl.label_text(x, mode='num'))

In [15]:
sl.batch_label_text(news['renderedContent'].iloc[:100])

In [21]:
news['date'] = pd.to_datetime(news['date'])

In [56]:
news['sentiment'] = np.random.randint(0, 3, size=news.shape[0])

In [63]:
# function to obtain news from certain period of time
def get_news(timestamp, period='24h'):
    end_datetime = timestamp
    if 'h' in period[-1]:
        hours = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(hours, unit='hours')
    elif 'm' in period[-1]:
        minutes = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(minutes, unit='minutes')
    elif 'd' in period[-1]:
        days = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(days, unit='days')
    elif 'w' in period[-1]:
        weeks = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(weeks, unit='W')
    else:
        assert False, "Incorrect entry for period"
    
    filter_bool = (news['date'] >= start_datetime) & (news['date'] <= end_datetime)
    return news[filter_bool]['renderedContent'].values

In [64]:
# function to obtain news sentiment from certain period of time
def get_news_sentiment(timestamp, period='24h'):
    end_datetime = timestamp
    if 'h' in period[-1]:
        hours = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(hours, unit='hours')
    elif 'm' in period[-1]:
        minutes = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(minutes, unit='minutes')
    elif 'd' in period[-1]:
        days = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(days, unit='days')
    elif 'w' in period[-1]:
        weeks = int(period[:-1])
        start_datetime = end_datetime - pd.Timedelta(weeks, unit='W')
    else:
        assert False, "Incorrect entry for period"
    
    filter_bool = (news['date'] >= start_datetime) & (news['date'] <= end_datetime)
    sentiment_scores = list(news[filter_bool]['sentiment'].values)
    return len(sentiment_scores), sentiment_scores.count(0), sentiment_scores.count(1), sentiment_scores.count(2)

In [70]:
test_datetime = datetime(2021, 1, 5, 9, 15, 0, tzinfo=pytz.timezone('US/Eastern'))

get_news(pd.to_datetime(test_datetime))

array(['Train services set to be reduced in lockdown bbc.in/3rVLlIp',
       'Tui and Thomas Cook cancel bookings as lockdowns begin bbc.in/3ncd4B7',
       'Paperchase on the brink of administration bbc.in/38g3Pvf',
       'Government offers firms £4.6bn in new lockdown grants bbc.in/3ogE9V0',
       "Morrisons' sales boosted by early Christmas shopping bbc.in/3ofTd5h",
       'On #WakeUpToMoney with @seanfarrington \n➡️CBI President, Lord Bilimoria, calls for more financial support for beleaguered businesses\n➡️How Scottish businesses are responding to the new restrictions\n➡️The business of vaccine manufacturing\n\n🎧Listen: bbc.in/3rVYqBh https://t.co/BXCiJrDbAR',
       'Advert of man eating bat sandwich investigated bbc.in/3rV2rpW',
       'NYSE scraps plans to delist Chinese telecoms giants bbc.in/35cyUOB',
       'Supermarket websites feel the strain of new lockdown bbc.in/38YPbru',
       'Amazon plots a course into the healthcare industry bbc.in/3ojgeEy',
       "UK 'cannot du

In [68]:
get_news_sentiment(pd.to_datetime(test_datetime), '1d')

(460, 153, 159, 148)